Importing necessary Libraries

In [1]:
import sagemaker
import boto3 #to read S3 buckets
from sagemaker.amazon.amazon_estimator import image_uris #to access xgboost model from Sagemaker
from sagemaker.session import Session 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
bucket_name = 'astonbankapplication' #change the name of the bucket to a unique name
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-2


In [ ]:
s3 = boto3.resource('s3', region_name=my_region)
try:
    if my_region == 'us-east-2':
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': 'us-east-2'
            }
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

### S3 Bucket on AWS Console

![Bucket](buckets.jpg)

In [3]:
#setting an output path
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://astonbankapplication/xgboost-as-a-built-in-algo/output


#### Downloading The Dataset And Storing in S3

In [4]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [5]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [6]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [7]:
### Saving Train And Test Into Buckets
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [8]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

Check your S3 bucket if the files have been uploaded successfully

### Files in Jupyter Notebook Instance

![xgboost_file](xgboost_file.jpg)

![Files](train_test_files.jpg)

### Building Models Xgboot- Inbuilt Algorithm

In [9]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.7-1'
)

In [10]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [11]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [12]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-22-23-33-53-927


2024-10-22 23:33:58 Starting - Starting the training job...
2024-10-22 23:34:12 Starting - Preparing the instances for training...
2024-10-22 23:34:42 Downloading - Downloading the training image...
2024-10-22 23:35:28 Training - Training image download completed. Training in progress....
2024-10-22 23:35:58 Uploading - Uploading generated training model.[2024-10-22 23:35:50.910 ip-10-0-129-238.us-east-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-10-22 23:35:50.933 ip-10-0-129-238.us-east-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-10-22:23:35:51:INFO] Imported framework sagemaker_xgboost_container.training
[2024-10-22:23:35:51:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-10-22:23:35:51:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-22:23:35:51:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-10-22:23:35:51:INFO] Dete

![training](Training_job.jpg)

### Output on AWS Console

![Output](output_train_test_files.jpg)

### Deploy Machine Learning Model As Endpoints

In [13]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-22-23-36-40-681
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-10-22-23-36-40-681
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-10-22-23-36-40-681


-------!

#### Prediction of the Test Data

In [14]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.array([float(x) for x in predictions.strip().split('\n')]) # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [15]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [16]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



#### Deleting The Endpoints

In [17]:
sagemaker.Session().delete_endpoint(endpoint_name=xgb_predictor.endpoint_name)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-10-22-23-36-40-681


[{'ResponseMetadata': {'RequestId': 'JTZRC04BXB7GC2AE',
   'HostId': 'vaXjZ80rOgq+YCJAUx+Tm3SU73u5vi98kfJsg+3EYEs/QFShDuz+7GcdyhjwJS7eiOC3BEv3sayTNX59Ey40QE+jYr4HBYoGLLJA7M2yHzw=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'vaXjZ80rOgq+YCJAUx+Tm3SU73u5vi98kfJsg+3EYEs/QFShDuz+7GcdyhjwJS7eiOC3BEv3sayTNX59Ey40QE+jYr4HBYoGLLJA7M2yHzw=',
    'x-amz-request-id': 'JTZRC04BXB7GC2AE',
    'date': 'Tue, 22 Oct 2024 23:40:44 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-10-22-23-33-53-927/debug-output/index/000000000/000000000040_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-10-22-23-33-53-927/debug-output/index/000000000/000000000020_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-10-22-23-33-53-927/debug-output/